In [1]:
import pandas as pd
import pickle
import requests
import time
from bs4 import BeautifulSoup

In [2]:
model = pickle.load(open('XGBoostmodel.pkl','rb'))

In [3]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    # closed = 'left' is take the current week out
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [35]:

def get_data(team_name, opponent_name, home, time_start, day):
    standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    name = [l.split('/')[-1].replace('-Stats', '').replace('-',' ') for l in links]
    df = pd.DataFrame({'team': name, 'link': links})
    team_url = df[df['team'] == team_name]['link'].values[0]

    team_data = requests.get(f'https://fbref.com{team_url}')
    matches = pd.read_html(team_data.text, match='Scores & Fixtures')[0]

    soup = BeautifulSoup(team_data.text)
    links = [l.get('href') for l in soup.find_all('a')]
    links = [l for l in links if l and '/all_comps/shooting/' in l]
    data = requests.get(f'https://fbref.com{links[0]}')
    shooting = pd.read_html(data.text, match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    # Because some team don't have shooting stat then when merge it error
    try:
        team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')
    except ValueError:
        # nothing to do
        pass

    # only want teamd data in PremierLeauge compettition
    team_data = team_data[team_data['Comp'] == 'Premier League']
    team_data['Season'] = 2024
    team_data['Team'] = team_name
    team_data.columns = [c.lower() for c in team_data.columns]

    # from parameter
    opponent_code_df = pd.read_csv('team_code.csv')
    opp_code = opponent_code_df[opponent_code_df['opponent'] == opponent_name]['opp_code'].values[0]
    venue_Home = 1 if home else 0
    mapping_day = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}
    day_code = mapping_day[day]
    hour = time_start

    cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'xg', 'xga']
    new_cols = [f'{c}_rollinig' for c in cols]

    team_data['date'] = pd.to_datetime(team_data['date'])
    team_data = team_data.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
    team_data = team_data.droplevel('team')
    team_data.index = range(team_data.shape[0])

    team_data = team_data.tail(1)
    team_data['opp_code'] = opp_code
    team_data['venue_Home'] = venue_Home
    team_data['day_code'] = day_code
    team_data['hour'] = hour

    cols_to_return = ['venue_Home', 'opp_code', 'hour', 'day_code'] + new_cols

    return team_data[cols_to_return]



In [36]:
def predict_match(model, match_data_team1, match_data_team2):
    inverse_mapping = { 1: 'Win', 0: 'Not win'}
    match_data_to_predict = pd.concat([match_data_team1, match_data_team2], axis=0)
    predicted = model.predict(match_data_to_predict)
    predicted_team1 = inverse_mapping[predicted[0]]
    predicted_team2 = inverse_mapping[predicted[1]]
    return predicted_team1, predicted_team2

In [37]:
# I will predict ManUTD vs Arsenal at emirates stadium 4th December 2024
match_data_mutd = get_data('Manchester United', 'Arsenal', home=False, time_start=20, day='Thu')
time.sleep(5)
match_data_arsenal = get_data('Arsenal', 'Manchester Utd', home=True, time_start=20, day='Thu')

C:\Users\phumi\AppData\Local\Temp\ipykernel_14128\3794418551.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_data.text, match='Scores & Fixtures')[0]
C:\Users\phumi\AppData\Local\Temp\ipykernel_14128\3794418551.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\phumi\AppData\Local\Temp\ipykernel_14128\3794418551.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this

In [38]:
match_data_mutd

,venue_Home,opp_code,hour,day_code,gf_rollinig,ga_rollinig,sh_rollinig,sot_rollinig,dist_rollinig,fk_rollinig,pk_rollinig,pkatt_rollinig,xg_rollinig,xga_rollinig
9,0,0,20,3,1.666667,0.666667,11.333333,3.333333,18.4,0.666667,0.333333,0.333333,1.2,1.1


In [39]:
match_data_arsenal.shape

(1, 14)

In [40]:
predicted_mutd_labels, predicted_ars_labels = predict_match(model, match_data_mutd, match_data_arsenal)

print(f'Manchester will: {predicted_mutd_labels} Arsenal')
print(f'Arsenal will: {predicted_ars_labels} Manchester United')

Manchester will: Not win Arsenal
Arsenal will: Win Manchester United


In [4]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}
response = requests.get('https://fbref.com', headers=headers)
print(response.status_code)  # Should be 200 if allowed

200
